In [1]:
# Import necessary libraries
import os 
import dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from typing import Dict
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [2]:
# Load environment variables
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Setup the chat model
Model = 'llama3-8b-8192'
chat = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name=Model)

# Define the prompt templates
system_prompt = "You are a helpful assistant equipped with a specific knowledge base. Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say 'I don't know'"
human_prompt = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human_prompt)])

# Build the conversational chain
chain = prompt | chat

# Load the document from the web
loader = WebBaseLoader("https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection")
data = loader.load()

# Split the document text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [3]:
# Create the embedding model
embedding = OpenAIEmbeddings()

# Define the persistence directory
persist_dir = 'Embeddings'

# Create the vector store and specify the persistence directory
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_dir)

# Load the vector store
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Define the retriever
retriever = vectorstore.as_retriever(k=4)

In [4]:
# Define the query transforming retriever chain
query_transform_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.")
])

query_transforming_retriever_chain = RunnableBranch(
    (lambda x: len(x.get("messages", [])) == 1,
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

# Define the question answering prompt
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="messages")
])

# Create the document chain
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

# Combine the chains into the conversational retrieval chain
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [5]:
# Initialize the chat message history
demo_ephemeral_chat_history = ChatMessageHistory()

# Add initial user message
demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

# Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})

# Extract and print only the answer
print(response['answer'])

The United Nations Guidelines for Consumer Protection aim to promote fair and transparent business practices, protect consumers' rights, and ensure their safety and well-being. The guidelines cover various aspects of consumer protection, including:

1. Information and Education: Providing consumers with accurate and timely information about products and services, as well as education on consumer rights and responsibilities.
2. Protection from Unfair Practices: Prohibiting unfair and deceptive business practices, such as false advertising, bait-and-switch tactics, and hidden fees.
3. Product Safety and Quality: Ensuring that products meet safety and quality standards, and that manufacturers and sellers are held accountable for defects and recalls.
4. Consumer Redress: Providing mechanisms for consumers to seek redress for grievances, such as complaints procedures, mediation, and arbitration.
5. Consumer Education and Awareness: Promoting consumer education and awareness through public c

In [6]:
# Add the AI's answer to the chat history
demo_ephemeral_chat_history.add_ai_message(response["answer"])

# Continue the conversation
demo_ephemeral_chat_history.add_user_message("tell me more about that!")
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

The United Nations Guidelines for Consumer Protection are a set of principles and recommendations aimed at promoting fair and transparent business practices, protecting consumers' rights, and ensuring their safety and well-being. Here are some additional details:

1. Principles: The guidelines are based on seven core principles:
	* Transparency: Businesses should be transparent in their dealings with consumers, including clear labeling, pricing, and advertising practices.
	* Accountability: Businesses should be held accountable for their actions, including penalties for non-compliance and enforcement of consumer protection laws.
	* Consumer Empowerment: Consumers should be empowered to make informed decisions, negotiate, and advocate for their rights.
	* Protection: Consumers should be protected from unfair and deceptive business practices, including false advertising, bait-and-switch tactics, and hidden fees.
	* Information: Consumers should have access to accurate and timely informat

In [7]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("thanks for that, so how is the consumer protection guaranteed")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

According to the United Nations Guidelines for Consumer Protection, consumer protection is guaranteed through a combination of measures, including:

1. **Laws and Regulations**: Governments are encouraged to enact and enforce laws and regulations that protect consumers from unfair and deceptive business practices.
2. **Consumer Protection Agencies**: Governments are encouraged to establish and empower consumer protection agencies to investigate complaints, conduct investigations, and take enforcement actions against violators.
3. **Consumer Education and Awareness**: Governments and businesses are encouraged to educate consumers about their rights and responsibilities, as well as the importance of consumer protection.
4. **Product Safety and Quality**: Governments are encouraged to establish and enforce standards for product safety and quality, and to ensure that products meet these standards.
5. **Redress Mechanisms**: Governments are encouraged to establish mechanisms for consumers t

In [8]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("tell me about chelsea football club")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

I'm afraid I'm stuck on the topic of consumer protection guidelines!


In [9]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("tell me about child stunting")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

I apologize, but there is no information about "child stunting" in the provided context. It seems that the context is about the United Nations Guidelines for Consumer Protection, and there is no mention of "child stunting". If you meant to ask about something else, please feel free to rephrase your question!
